# Laboratorio 7 - Deep Learning

## Autores

- Angel Higueros 20460
- Fredy Velasquez 201011



## Task 1 - Práctica
Considere las arquitecturas conversadas durante la clase, con ello realice una implementación de dos arquitecturas
usando PyTorch
1. Implemente la arquitectura de LeNet-5 para resolver el problema de clasificación del daset de dígitos escritos a mano llamado mnist dataset
2. Implemente la arquitectura de AlexNet para resolver el problema de clasificación usando el dataset de imagenes llamado CIFAR10 dataset.

Para cada implementación defina y justifique (dentro del notebook) una métrica de desempeño. Además responda
(en su notebook) 


Recuerde justificar y/o expandir su respuesta:
- ¿Cuál es la diferencia principal entre ambas arquitecturas?
- ¿Podría usarse LeNet-5 para un problema como el que resolvió usando AlexNet? ¿Y viceversa?
- Indique claramente qué le pareció más interesante de cada arquitectura




## LeNet-5

In [1]:
#Imports necesarios
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [2]:
# Definir una transformacion para normalizar las imágenes
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Descargar el conjunto de datos y aplicar transformaciones
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# Definir DataLoader para cargar los datos
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Definir la arquitectura de LeNet-5
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Definir una función de entrenamiento y prueba, calcular la precision como metrica
def train(model, train_loader, criterion, optimizer):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    return total_loss / len(train_loader), correct / total

def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# Entrenar la red y evaluar la precisión
lenet5 = LeNet5()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lenet5.parameters(), lr=0.001)

for epoch in range(10):
    train_loss, train_acc = train(lenet5, train_loader, criterion, optimizer)
    test_acc = test(lenet5, test_loader)
    print(f'Epoch {epoch + 1}/{10}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')


100%|██████████| 9912422/9912422 [00:00<00:00, 24913671.78it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 20534954.03it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 4482203.05it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 2704888.37it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Epoch 1/10: Train Loss: 0.2490, Train Acc: 0.9239, Test Acc: 0.9788
Epoch 2/10: Train Loss: 0.0653, Train Acc: 0.9797, Test Acc: 0.9812
Epoch 3/10: Train Loss: 0.0488, Train Acc: 0.9846, Test Acc: 0.9866
Epoch 4/10: Train Loss: 0.0382, Train Acc: 0.9885, Test Acc: 0.9877
Epoch 5/10: Train Loss: 0.0328, Train Acc: 0.9894, Test Acc: 0.9889
Epoch 6/10: Train Loss: 0.0270, Train Acc: 0.9915, Test Acc: 0.9883
Epoch 7/10: Train Loss: 0.0224, Train Acc: 0.9927, Test Acc: 0.9910
Epoch 8/10: Train Loss: 0.0199, Train Acc: 0.9937, Test Acc: 0.9903
Epoch 9/10: Train Loss: 0.0179, Train Acc: 0.9941, Test Acc: 0.9889
Epoch 10/10: Train Loss: 0.0153, Train Acc: 0.9952, Test Acc: 0.9891


## Task 2 - Teoría
Responda claramente y con una extensión adecuada las siguientes preguntas:
1. Investigue e indique en qué casos son útiles las siguientes arquitecturas, agregue imagenes si esto le ayuda a una mejor comprensión
- GoogleNet (Inception)

La arquitectura GoogleNet es útil en casos donde se busca una red profunda con una computación eficiente. Su característica distintiva es el uso de módulos de "inception" que consisten en múltiples convoluciones y concatenaciones de características de diferentes tamaños de kernel en paralelo. Esto permite que la red capture características a diferentes escalas y resoluciones, lo que es útil en tareas de clasificación de imágenes, detección de objetos y segmentación semántica. GoogleNet es especialmente efectiva en la optimización de uso de recursos computacionales.

- DenseNet (Densely Connected Convolutional Networks)

DenseNet es útil cuando se busca una arquitectura que promueva conexiones densas entre capas. En lugar de tener conexiones dispersas entre capas, como en las CNN tradicionales, DenseNet conecta todas las capas entre sí. Esto fomenta el flujo de información a lo largo de la red y facilita el aprendizaje de características complejas y la mitigación del problema de desvanecimiento de gradientes. DenseNet es beneficioso para la clasificación de imágenes, detección de objetos y segmentación de imágenes.

- MobileNet

MobileNet es útil en casos donde se requiere una red ligera y eficiente en términos de recursos computacionales, como en aplicaciones móviles o dispositivos con recursos limitados. Esta arquitectura utiliza convoluciones separables en profundidad (depthwise separable convolutions) para reducir la cantidad de operaciones y parámetros, sin sacrificar demasiado la precisión. MobileNet es ideal para la clasificación de imágenes en dispositivos móviles y la detección en tiempo real.

- EfficientNet

EfficientNet es útil en situaciones donde se busca un equilibrio óptimo entre el rendimiento y la eficiencia computacional. Esta arquitectura utiliza un enfoque de búsqueda en escalas compuestas para encontrar el tamaño de modelo adecuado para una tarea dada, escalando la profundidad, el ancho y la resolución de la red de manera equilibrada. EfficientNet ha demostrado ser altamente eficiente en términos de rendimiento de precisión en una amplia gama de tareas de visión por computadora, incluida la clasificación de imágenes y la detección de objetos.

2. ¿Cómo la arquitectura de transformers puede ser usada para image recognition?

La arquitectura de Transformers, que se originó en el procesamiento de lenguaje natural (NLP), también se puede utilizar para el reconocimiento de imágenes. Esto se logra mediante la adaptación de la arquitectura Transformer original para el procesamiento de secuencias de texto a la tarea de procesar matrices de imágenes.

El enfoque principal para usar Transformers en el reconocimiento de imágenes se llama "Transformers para visión" o "Vision Transformers". La idea central es tratar una imagen como una secuencia de parches (patches), donde cada parche es una porción de la imagen. Cada parche se representa como un vector y se pasa a través de la arquitectura Transformer.

Usar Transformer con imagenes: 

División de la imagen en parches: La imagen se divide en una cuadrícula de parches, y cada parche se transforma en un vector de características.
Posición de codificación: Se agrega información de posición a los vectores de características de los parches, similar a cómo se manejan las secuencias en NLP.
Capas de atención multiatención: Se aplican capas de atención multiatención (self-attention) para permitir que los parches se relacionen entre sí.
Redes completamente conectadas: Después de las capas de atención, se utilizan capas de redes neuronales completamente conectadas para realizar la clasificación.
Esta arquitectura ha demostrado ser efectiva en una variedad de tareas de visión por computadora, incluido el reconocimiento de objetos, la detección de objetos y la segmentación semántica. Un ejemplo de implementación de Vision Transformer (ViT) en PyTorch se encuentra en la biblioteca "PyTorch Vision" de Hugging Face.
